# Flights Dataset Preprocessing and feature Engineering

In [13]:
%AddJar file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar

Using cached version of Emiasd-Flight-Data-Analysis.jar


error: error while loading FlightDataLoader, class file '/tmp/toree-tmp-dir14029213483107734240/toree_add_jars/Emiasd-Flight-Data-Analysis.jar(com/flightdelay/data/loaders/FlightDataLoader.class)' has location not matching its contents: contains class FlightDataLoader


Using cached version of Emiasd-Flight-Data-Analysis.jar


In [41]:
import org.apache.spark.sql.SparkSession
import com.flightdelay.config.{AppConfiguration, ConfigurationLoader}
import com.flightdelay.data.loaders.FlightDataLoader

//Env Configuration
val args: Array[String] = Array("jupyter")

val spark = SparkSession.builder()
  .config(sc.getConf)
  .getOrCreate()

// Rendre la session Spark implicite
implicit val session = spark
implicit val configuration: AppConfiguration = ConfigurationLoader.loadConfiguration(args)

val rawFlightsPath = s"${configuration.common.output.basePath}/common/data/processed_flights.parquet"
val flightsDF = spark.read.parquet(rawFlightsPath)

lastException = null
args = Array(jupyter)
spark = org.apache.spark.sql.SparkSession@16b09d4a
session = org.apache.spark.sql.SparkSession@16b09d4a
configuration = AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/201201hourly.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileCo...


AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/201201hourly.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileCo...

In [42]:
flightsDF.count()

403884

In [43]:
flightsDF.printSchema

root
 |-- FL_DATE: date (nullable = true)
 |-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN_AIRPORT_ID: integer (nullable = true)
 |-- DEST_AIRPORT_ID: integer (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- ARR_DELAY_NEW: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- WEATHER_DELAY: double (nullable = true)
 |-- NAS_DELAY: double (nullable = true)
 |-- ORIGIN_WBAN: string (nullable = true)
 |-- ORIGIN_TIMEZONE: integer (nullable = true)
 |-- DEST_WBAN: string (nullable = true)
 |-- DEST_TIMEZONE: integer (nullable = true)
 |-- UTC_CRS_DEP_TIME: string (nullable = true)
 |-- UTC_ARR_TIME: string (nullable = true)
 |-- UTC_ARR_DATE: date (nullable = true)
 |-- CRS_ARR_TIME: string (nullable = true)
 |-- CRS_ARR_DATE: date (nullable = true)
 |-- feature_arrival_hour: integer (nullable = true)
 |-- feature_utc_arrival_hour: integer (nullable = true)
 |-- feature_utc_arrival

In [44]:
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.ml.feature.{StringIndexer, OneHotEncoder, VectorAssembler}
import scala.util.Try

In [45]:
flightsDF.count()

403884

# Flight Delay Metrics

In [67]:
import org.apache.spark.sql.functions._

val delayColumns = Seq(
  "FL_DATE",
  "OP_CARRIER_FL_NUM",
  "ORIGIN_AIRPORT_ID",
  "CRS_DEP_TIME",
  "WEATHER_DELAY",
  "NAS_DELAY",
  "ARR_DELAY_NEW",

  /**"label_is_early",
  "label_is_on_time",
    
  "label_arr_delay_filled",
  "label_nas_delay_filled",
  "label_weather_delay_filled",
    
  "label_weather_delay_was_missing",
  "label_nas_delay_was_missing",
  "label_arr_delay_was_missing",**/
  
  "label_total_weather_nas_delay",

  /**"label_has_nas_delay",
  "label_has_weather_delay",
  "label_has_any_weather_nas_delay",**/
    
  "label_is_delayed_15min",
  /**"label_is_delayed_30min",
  "label_is_delayed_60min",
  "label_is_delayed_45min",
  "label_is_delayed_90min",
  
  "feature_avg_delay",
  "feature_num_previous_flights",
  "feature_stddev_delay",
  "feature_max_delay",
  "feature_min_delay",**/

  "feature_proportion_delayed_15min",    
  /**"feature_proportion_delayed_30min",
  "feature_proportion_delayed_60min",
  "feature_proportion_delayed_45min",
  "feature_proportion_delayed_90min"**/
)

val df1 = flightsDF
  .select(delayColumns.head, delayColumns.tail: _*)
  /**.filter(
    col("FL_DATE") === "2013-03-01" &&
    col("OP_CARRIER_FL_NUM") === 362 &&  
    col("WEATHER_DELAY").isNotNull && col("WEATHER_DELAY") > 0 &&
    col("NAS_DELAY").isNotNull && col("NAS_DELAY") > 0
  )**/

df1.show(5)


delayColumns: Seq[String] = List(FL_DATE, OP_CARRIER_FL_NUM, ORIGIN_AIRPORT_ID, CRS_DEP_TIME, WEATHER_DELAY, NAS_DELAY, ARR_DELAY_NEW, label_total_weather_nas_delay, label_is_delayed_15min, feature_proportion_delayed_15min)
df1: org.apache.spark.sql.DataFrame = [FL_DATE: date, OP_CARRIER_FL_NUM: int ... 8 more fields]


+----------+-----------------+-----------------+------------+-------------+---------+-------------+-----------------------------+----------------------+--------------------------------+
|   FL_DATE|OP_CARRIER_FL_NUM|ORIGIN_AIRPORT_ID|CRS_DEP_TIME|WEATHER_DELAY|NAS_DELAY|ARR_DELAY_NEW|label_total_weather_nas_delay|label_is_delayed_15min|feature_proportion_delayed_15min|
+----------+-----------------+-----------------+------------+-------------+---------+-------------+-----------------------------+----------------------+--------------------------------+
|2013-07-31|              916|            14771|        2100|          0.0|      0.0|         16.0|                          0.0|                     1|            0.052345215759849906|
|2013-07-31|             6530|            14771|        1945|          0.0|      0.0|         20.0|                          0.0|                     1|            0.052345215759849906|
|2013-07-31|             6513|            14771|        1600|         

# BTS Data

In [29]:
val DEFAULT_DATE_FORMAT = "yyyymmdd"
val filePath = s"${configuration.common.data.basePath}/BTS-201303/T_ONTIME_REPORTING_201303.csv"

val btsDF = spark.read.format("csv")
  .option("header", "true")
  //.schema(expectedSchema)
  .option("timestampFormat", DEFAULT_DATE_FORMAT)
  .option("multiline", "true")
  .option("escape", "\"")
  .load(filePath)
  //.withColumn("FL_DATE", to_date(col("FL_DATE"), DEFAULT_DATE_FORMAT))

DEFAULT_DATE_FORMAT = yyyymmdd
filePath = /home/jovyan/work/data/BTS-201303/T_ONTIME_REPORTING_201303.csv
btsDF = [YEAR: string, QUARTER: string ... 71 more fields]


[YEAR: string, QUARTER: string ... 71 more fields]

In [30]:
btsDF.printSchema

root
 |-- YEAR: string (nullable = true)
 |-- QUARTER: string (nullable = true)
 |-- MONTH: string (nullable = true)
 |-- DAY_OF_MONTH: string (nullable = true)
 |-- DAY_OF_WEEK: string (nullable = true)
 |-- FL_DATE: string (nullable = true)
 |-- OP_UNIQUE_CARRIER: string (nullable = true)
 |-- OP_CARRIER_AIRLINE_ID: string (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- TAIL_NUM: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: string (nullable = true)
 |-- ORIGIN_AIRPORT_ID: string (nullable = true)
 |-- ORIGIN_AIRPORT_SEQ_ID: string (nullable = true)
 |-- ORIGIN_CITY_MARKET_ID: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- ORIGIN_CITY_NAME: string (nullable = true)
 |-- ORIGIN_STATE_ABR: string (nullable = true)
 |-- ORIGIN_STATE_FIPS: string (nullable = true)
 |-- ORIGIN_STATE_NM: string (nullable = true)
 |-- ORIGIN_WAC: string (nullable = true)
 |-- DEST_AIRPORT_ID: string (nullable = true)
 |-- DEST_AIRPORT_SEQ_ID: string (nullable = true)

In [66]:
import org.apache.spark.sql.functions._

val btsDelayColumns = Seq(
  "FL_DATE",
  //"OP_UNIQUE_CARRIER",
  //"OP_CARRIER_AIRLINE_ID",
  //"OP_CARRIER",
  //"TAIL_NUM",
  "OP_CARRIER_FL_NUM",  
  /**"ORIGIN_AIRPORT_ID",
  "ORIGIN_AIRPORT_SEQ_ID",
  "ORIGIN_CITY_MARKET_ID",
  "DEST_AIRPORT_ID",
  "DEST_AIRPORT_SEQ_ID",
  "DEST_CITY_MARKET_ID",**/
  "CRS_DEP_TIME",
  /**"DEP_TIME",
  "DEP_DELAY",
  "DEP_DELAY_NEW",
  "DEP_DEL15",
  "DEP_DELAY_GROUP",
  "DEP_TIME_BLK",
  "TAXI_OUT",
  "WHEELS_OFF",
  "WHEELS_ON",
  "TAXI_IN",
  "CRS_ARR_TIME",
  "ARR_TIME",**/
  "ARR_DELAY",
  "ARR_DELAY_NEW",
  "ARR_DEL15",
  /**"ARR_DELAY_GROUP",
  "ARR_TIME_BLK",
  "CANCELLED",
  "CANCELLATION_CODE",
  "DIVERTED",
  "CRS_ELAPSED_TIME",
  "ACTUAL_ELAPSED_TIME",
  "AIR_TIME",
  "FLIGHTS",
  "DISTANCE",
  "DISTANCE_GROUP",**/
  "CARRIER_DELAY",
  "WEATHER_DELAY",
  "NAS_DELAY",
  "SECURITY_DELAY",
  "LATE_AIRCRAFT_DELAY"
)

val df2 = btsDF
  .select(btsDelayColumns.head, btsDelayColumns.tail: _*)
  /**.filter(
    col("SECURITY_DELAY").isNotNull && col("SECURITY_DELAY") > 0 &&
    col("LATE_AIRCRAFT_DELAY").isNotNull && col("LATE_AIRCRAFT_DELAY") > 0
  )**/

df2.show(100)

btsDelayColumns: Seq[String] = List(FL_DATE, OP_CARRIER_FL_NUM, CRS_DEP_TIME, ARR_DELAY, ARR_DELAY_NEW, ARR_DEL15, CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY)
df2: org.apache.spark.sql.DataFrame = [FL_DATE: string, OP_CARRIER_FL_NUM: string ... 9 more fields]


+--------------------+-----------------+------------+---------+-------------+---------+-------------+-------------+---------+--------------+-------------------+
|             FL_DATE|OP_CARRIER_FL_NUM|CRS_DEP_TIME|ARR_DELAY|ARR_DELAY_NEW|ARR_DEL15|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|
+--------------------+-----------------+------------+---------+-------------+---------+-------------+-------------+---------+--------------+-------------------+
|3/1/2013 12:00:00 AM|             3697|        1136|     NULL|         NULL|     NULL|         NULL|         NULL|     NULL|          NULL|               NULL|
|3/1/2013 12:00:00 AM|             3697|        1255|     NULL|         NULL|     NULL|         NULL|         NULL|     NULL|          NULL|               NULL|
|3/1/2013 12:00:00 AM|             4105|        1520|     NULL|         NULL|     NULL|         NULL|         NULL|     NULL|          NULL|               NULL|
|3/1/2013 12:00:00 AM|            

# Jointure entre deux dataset

In [81]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

// Normalisation de la colonne FL_DATE dans df1
// Format actuel: "2013-03-11" -> déjà au bon format
val df1Normalized = df1.withColumn(
  "FL_DATE_normalized",
  col("FL_DATE") // Garder tel quel
)

//df1Normalized.show(10)

// Alternative: Normaliser df2 au format de df1 (plus simple)
val df2Normalized = df2.withColumn(
  "FL_DATE_normalized",
  // Convertir "3/1/2013 12:00:00 AM" en "2013-01-03"
  // Extraire la partie date avant l'espace
  regexp_extract(col("FL_DATE"), "^(\\d{1,2})/(\\d{1,2})/(\\d{4})", 0)
).withColumn(
  "FL_DATE_normalized",
  // Parser et reformater: M/d/yyyy -> yyyy-MM-dd
  date_format(
    to_date(col("FL_DATE_normalized"), "M/d/yyyy"),
    "yyyy-MM-dd"
  )
)
//df2Normalized.show(10)


// Méthode 1: Jointure avec SECURITY_DELAY > 0
val joinedDF1 = df1Normalized
  .join(
    df2Normalized.filter(col("WEATHER_DELAY") > 0),
    df1Normalized("FL_DATE_normalized") === df2Normalized("FL_DATE_normalized") &&
    df1Normalized("OP_CARRIER_FL_NUM") === df2Normalized("OP_CARRIER_FL_NUM") &&
    df1Normalized("CRS_DEP_TIME") === df2Normalized("CRS_DEP_TIME"),
    "inner"
  )
  .select(df2Normalized.columns.map(df2Normalized(_)): _*)
  .drop("FL_DATE_normalized") // Nettoyer la colonne temporaire de df1

joinedDF1.show(300000)

+--------------------+-----------------+------------+---------+-------------+---------+-------------+-------------+---------+--------------+-------------------+
|             FL_DATE|OP_CARRIER_FL_NUM|CRS_DEP_TIME|ARR_DELAY|ARR_DELAY_NEW|ARR_DEL15|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|
+--------------------+-----------------+------------+---------+-------------+---------+-------------+-------------+---------+--------------+-------------------+
|3/1/2013 12:00:00 AM|              507|        1130|    15.00|        15.00|     1.00|         0.00|        15.00|     0.00|          0.00|               0.00|
|3/1/2013 12:00:00 AM|             1556|        1215|    20.00|        20.00|     1.00|         0.00|        11.00|     0.00|          0.00|               9.00|
|3/1/2013 12:00:00 AM|              362|        1530|    20.00|        20.00|     1.00|         0.00|        20.00|     0.00|          0.00|               0.00|
|3/1/2013 12:00:00 AM|            

df1Normalized: org.apache.spark.sql.DataFrame = [FL_DATE: date, OP_CARRIER_FL_NUM: int ... 9 more fields]
df2Normalized: org.apache.spark.sql.DataFrame = [FL_DATE: string, OP_CARRIER_FL_NUM: string ... 10 more fields]
joinedDF1: org.apache.spark.sql.DataFrame = [FL_DATE: string, OP_CARRIER_FL_NUM: string ... 9 more fields]


In [72]:
val sampleDf1 = df1
  .select(delayColumns.head, delayColumns.tail: _*)
  .filter(
    col("FL_DATE") === "2013-03-01" &&
    col("OP_CARRIER_FL_NUM") === 100 &&  
    col("CRS_DEP_TIME") === 655
  )

sampleDf1.show(5)

+----------+-----------------+-----------------+------------+-------------+---------+-------------+-----------------------------+----------------------+--------------------------------+
|   FL_DATE|OP_CARRIER_FL_NUM|ORIGIN_AIRPORT_ID|CRS_DEP_TIME|WEATHER_DELAY|NAS_DELAY|ARR_DELAY_NEW|label_total_weather_nas_delay|label_is_delayed_15min|feature_proportion_delayed_15min|
+----------+-----------------+-----------------+------------+-------------+---------+-------------+-----------------------------+----------------------+--------------------------------+
|2013-03-01|              100|            13871|         655|          0.0|      4.0|         17.0|                          4.0|                     1|                             0.0|
+----------+-----------------+-----------------+------------+-------------+---------+-------------+-----------------------------+----------------------+--------------------------------+



sampleDf1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [FL_DATE: date, OP_CARRIER_FL_NUM: int ... 8 more fields]


In [75]:
val sampleDf2 = df2
  .select(btsDelayColumns.head, btsDelayColumns.tail: _*)
  .filter(
    col("FL_DATE") === "3/1/2013 12:00:00 AM" &&
    col("OP_CARRIER_FL_NUM") === 100 &&  
    col("CRS_DEP_TIME") === 655
  )

sampleDf2.show(5)

+--------------------+-----------------+------------+---------+-------------+---------+-------------+-------------+---------+--------------+-------------------+
|             FL_DATE|OP_CARRIER_FL_NUM|CRS_DEP_TIME|ARR_DELAY|ARR_DELAY_NEW|ARR_DEL15|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|
+--------------------+-----------------+------------+---------+-------------+---------+-------------+-------------+---------+--------------+-------------------+
|3/1/2013 12:00:00 AM|              100|        0655|    17.00|        17.00|     1.00|         0.00|         0.00|     4.00|         13.00|               0.00|
+--------------------+-----------------+------------+---------+-------------+---------+-------------+-------------+---------+--------------+-------------------+



lastException: Throwable = null
sampleDf2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [FL_DATE: string, OP_CARRIER_FL_NUM: string ... 9 more fields]
